In [ ]:
from src.trainer import train
import random
import numpy as np
import gymnasium as gym
import config

In [ ]:
agent = train()

Ep 10 reward -9.0 MA(100) 0.20 eps 0.990
Ep 20 reward -2.0 MA(100) 2.05 eps 0.980
Ep 30 reward -6.0 MA(100) 0.90 eps 0.970
Ep 40 reward 3.0 MA(100) 0.75 eps 0.961
Ep 50 reward -4.0 MA(100) 0.42 eps 0.951
Ep 60 reward -3.0 MA(100) -0.40 eps 0.942
Ep 70 reward -5.0 MA(100) -1.03 eps 0.932
Ep 80 reward -11.0 MA(100) -1.65 eps 0.923
Ep 90 reward -7.0 MA(100) -2.33 eps 0.914
Ep 100 reward -10.0 MA(100) -2.62 eps 0.905
q_table stats: min -2.578, mean -0.124, max 1.409
Ep 110 reward -12.0 MA(100) -3.34 eps 0.896
Ep 120 reward -5.0 MA(100) -4.40 eps 0.887
Ep 130 reward -6.0 MA(100) -4.76 eps 0.878
Ep 140 reward -3.0 MA(100) -5.33 eps 0.869
Ep 150 reward -6.0 MA(100) -5.60 eps 0.861
Ep 160 reward 3.0 MA(100) -5.25 eps 0.852
Ep 170 reward -4.0 MA(100) -4.79 eps 0.844
Ep 180 reward 4.0 MA(100) -4.12 eps 0.835
Ep 190 reward 6.0 MA(100) -3.54 eps 0.827
Ep 200 reward -3.0 MA(100) -3.16 eps 0.819
q_table stats: min -4.960, mean -0.375, max 2.436
Ep 210 reward 26.0 MA(100) -2.15 eps 0.810
Ep 220 rewar

c:\Users\Andrija Lukic\AppData\Local\Programs\Python\Python310\lib\site-packages\pygame\pkgdata.py:27: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Ep 510 reward 106.0 MA(100) 93.09 eps 0.600
Ep 520 reward 105.0 MA(100) 94.02 eps 0.594
Ep 530 reward 120.0 MA(100) 100.02 eps 0.588
Ep 540 reward 100.0 MA(100) 101.71 eps 0.583
Ep 550 reward 97.0 MA(100) 102.28 eps 0.577
Ep 560 reward 116.0 MA(100) 104.16 eps 0.571
Ep 570 reward 123.0 MA(100) 106.11 eps 0.565
Ep 580 reward 110.0 MA(100) 108.66 eps 0.560
Ep 590 reward 114.0 MA(100) 111.71 eps 0.554
Ep 600 reward 115.0 MA(100) 112.30 eps 0.549
q_table stats: min -1.686, mean 12.751, max 57.753
Ep 610 reward 130.0 MA(100) 114.85 eps 0.543
Ep 620 reward 136.0 MA(100) 118.15 eps 0.538
Ep 630 reward 158.0 MA(100) 122.68 eps 0.532
Ep 640 reward 161.0 MA(100) 130.39 eps 0.527
Ep 650 reward 148.0 MA(100) 136.53 eps 0.522
Ep 660 reward 182.0 MA(100) 144.08 eps 0.517
Ep 670 reward 162.0 MA(100) 150.82 eps 0.512
Ep 680 reward 189.0 MA(100) 156.54 eps 0.506
Ep 690 reward 171.0 MA(100) 161.75 eps 0.501
Ep 700 reward 190.0 MA(100) 167.98 eps 0.496
q_table stats: min -1.686, mean 16.977, max 84.902
E

In [ ]:
def test(fql_agent, num_episodes=100, render=False, seed=None, max_steps=None, verbose=True):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    env = gym.make(config.ENV_NAME)
    max_steps = max_steps or getattr(config, "MAX_STEPS_PER_EPISODE", 1000)

    prev_eps = getattr(fql_agent, "epsilon", None)
    if prev_eps is not None:
        fql_agent.epsilon = 0.0

    episode_rewards = []
    try:
        for ep in range(1, num_episodes + 1):
            if seed is not None:
                obs, _ = env.reset(seed=seed + ep)
            else:
                obs, _ = env.reset()

            total_reward = 0.0
            terminated = truncated = False
            steps = 0
            cart_pos, cart_vel, pole_angle, pole_ang_vel = obs
            action = fql_agent.get_initial_action([float(pole_angle), float(pole_ang_vel)])

            while not (terminated or truncated) and steps < max_steps:
                obs, reward, terminated, truncated, _ = env.step(int(action))
                total_reward += reward
                steps += 1

                if terminated or truncated:
                    break

                _, _, pole_angle, pole_ang_vel = obs
                action = fql_agent.get_action([float(pole_angle), float(pole_ang_vel)])

            episode_rewards.append(total_reward)

            if verbose and (ep == 1 or ep % 10 == 0 or ep == num_episodes):
                recent_ma = float(np.mean(episode_rewards[-100:]))
                print(f"[Test] Ep {ep}/{num_episodes} reward {total_reward:.1f} recent_MA({min(100,len(episode_rewards))}) {recent_ma:.2f}")

    finally:
        if prev_eps is not None:
            fql_agent.epsilon = prev_eps
        env.close()

    stats = {
        "mean": float(np.mean(episode_rewards)),
        "std": float(np.std(episode_rewards)),
        "min": float(np.min(episode_rewards)),
        "max": float(np.max(episode_rewards)),
        "median": float(np.median(episode_rewards)),
    }

    if verbose:
        print("=== Evaluation summary ===")
        print(f"episodes: {len(episode_rewards)}  mean: {stats['mean']:.2f}  std: {stats['std']:.2f}  min: {stats['min']:.1f}  max: {stats['max']:.1f}")

    return episode_rewards, stats

In [4]:
test_rewards, test_stats = test(agent[0], num_episodes=50, render=True, seed=42)

[Test] Ep 1/50 reward 500.0 recent_MA(1) 500.00
[Test] Ep 10/50 reward 500.0 recent_MA(10) 500.00
[Test] Ep 20/50 reward 500.0 recent_MA(20) 500.00
[Test] Ep 30/50 reward 500.0 recent_MA(30) 500.00
[Test] Ep 40/50 reward 500.0 recent_MA(40) 500.00
[Test] Ep 50/50 reward 500.0 recent_MA(50) 500.00
=== Evaluation summary ===
episodes: 50  mean: 500.00  std: 0.00  min: 500.0  max: 500.0


In [6]:
np.save(f"q_table_best_ep{3500}.npy", agent[0].q_table)